## Pre-processing & Modeling

In [2]:
import pandas as pd
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
!pip install scikit-learn

In [51]:
# Load the data
df = pd.read_csv('clean_perfume_data.csv')

In [5]:
# Vectorize the notes column
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['notes'].values.astype('U'))

In [6]:
# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [67]:
# Function to recommend perfumes based on user input
def recommend_perfumes(perfume_name, cosine_sim=cosine_sim, df=df):
    # Convert user input to lowercase
    perfume_name = perfume_name.lower()
    
    # Find indices of matching perfumes
    matching_indices = df[df['perfume'].str.lower() == perfume_name].index
    if len(matching_indices) == 0:
        return "Perfume not found in the dataset. Please enter another perfume name."
    else:
        idx = matching_indices[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Exclude user input perfume from recommended list
        recommended_indices = [i[0] for i in sim_scores if i[0] != idx]
        recommended_indices = recommended_indices[:10]  # Top 10 similar perfumes
        
        recommendations = df.iloc[recommended_indices][['brand', 'perfume', 'notes']]
        return recommendations

In [69]:
# simulate Streamlit app behavior
def main(perfume_name):
    recommendations = recommend_perfumes(perfume_name, cosine_sim, df)
    print("Recommended Perfumes:")
    print(recommendations)

# Simulate Streamlit user input and button click
perfume_name = 'hareem al sultan'  
main(perfume_name)

Recommended Perfumes:
                        brand                             perfume  \
31914                  Eudora                            S Priv e   
11693      David LISS Parfums                  Bergamote Chocolat   
22374                    Pupa                       Puposhka Gold   
25063           The Body Shop                             Amorito   
14976      Le Bel Aujourd hui                       Brun d Epices   
3307        Demeter Fragrance                      Dark Chocolate   
24295         Laurelle London     Les Couleurs No 4 Hot Chocolate   
7869   Sarah Horowitz Parfums                      Perfect Sunset   
24291         Laurelle London  Les Couleurs No 2 Chocolate Orange   
21727                 Missoni     Missoni Colori Missoni Gianduia   

                                                   notes  
31914  mandarin orange, floral notes, dark chocolate,...  
11693                           bergamot, dark chocolate  
22374           , amber, musk, vanilla, d

In [ ]:
# Streamlit app
def main():
    st.title('Fragrance Recommender System')
    st.sidebar.title('User Input')

    # User input
    perfume_name = st.sidebar.text_input('Enter a perfume name')  
    if st.sidebar.button('Get Recommendations'):
        recommendations = recommend_perfumes(perfume_name)
        st.subheader('Recommended Perfumes')
        st.write(recommendations)

if __name__ == '__main__':
    main()